In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import pandas_udf
from pyspark.sql import SparkSession

In [ ]:
property_data = pd.read_csv("data/daft_from_page_0_till_page_987_by_20.csv", index_col=["Unnamed: 0"])

In [213]:
property_data.to_csv("data/cleaned_data.csv", index=False)

In [140]:
property_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19845 entries, 0 to 19844
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Bed            16705 non-null  float64
 1   Bath           16435 non-null  float64
 2   m²             15458 non-null  float64
 3   Price          19845 non-null  object 
 4   Address        19845 non-null  object 
 5   Property_type  19833 non-null  object 
 6   New_build      19845 non-null  object 
 7   Page           19845 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.4+ MB


## Insights

Features that contain null values:
1. Bed and Bath
    1. If property_type == "site": Bed = 0 
    2. else: drop row or try impute the value.
2. m²
    1. Maybe try to impute the value ~4,000 null
3. Property_type
    1. Look into, might drop rows


In [144]:
property_data.describe().astype(int).drop(columns=["Page"])

,Bed,Bath,m²
count,16705,16435,15458
mean,3,2,28133
std,1,1,2372859
min,1,1,0
25%,3,1,90
50%,3,2,128
75%,4,3,268
max,40,32,263040751


### Insights

Some extream values, may need to investigate outliers.

In [145]:
property_data.nunique()

Bed                 25
Bath                23
m²                 994
Price              801
Address          18799
Property_type       11
New_build            2
Page               984
dtype: int64

### Insights

1. Price didn't come in the last section and has a dtype of object.
    2. May need to clean data.
2. There are 11 property types, maybe look into reducing this.

In [138]:
property_data["Property_type"].value_counts()

Detached          6000
Semi-D            3982
Site              3070
Terrace           2352
Apartment         2075
End of Terrace     929
Bungalow           869
Townhouse          248
Duplex             223
Studio               3
House                1
Name: Property_type, dtype: int64

I will look into hierarchical structure.  
People looking to live in a Studio may also be interested in Apartments,
and people looking a Semi-D may also be interested in Detached.